<a href="https://colab.research.google.com/github/yajima-yasutoshi/DataMinig/blob/main/20231213/%E5%88%86%E9%A1%9E2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データマイニング第12回（2023/12/13）

#本日の講義の目的

機械学習の準備として、
ロジスティック回帰を例にして、予測精度を向上させるテクニックに関して
詳しく説明を行う。

1. 説明変数にカテゴリ変数を用いる

1. ハイパーパラメータチューニング


## 準備


In [ ]:
# インストール
!pip install japanize-matplotlib

## 必要なライブラリーのインポート

In [ ]:
# 必要なライブラリをインポート
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
# 必要なライブラリのインポート
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 分類（ロジスティック回帰）に必要なライブラリー
from sklearn.linear_model import LogisticRegression

# その他必要なライブラリー
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

### 【参考】ロジスティック回帰ライブラリー
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

# Titanic データを使った例


In [ ]:
# データを読み込む
# 変数名を data とした
data = sns.load_dataset("titanic") #タイタニックのデータ
data.info()
data.head(5)

##データの説明

列名 | 型 | 意味
---  | --- | --
survived	| カテゴリ | 生存フラグ（0=死亡、1=生存）
pclass	|  カテゴリ |チケットクラス（1stクラス、2ndクラス、3rdクラス）
sex	|  カテゴリ |性別（male=男性、female＝女性）
sge	|  数値 |年齢
sibsp	|  数値 |タイタニックに同乗している兄弟/配偶者の数
parch	|  数値 |タイタニックに同乗している親/子供の数
fare	|  数値 |料金
embarked	|  カテゴリ |出港地（タイタニックへ乗った港）(C=Cherbourg、Q=Queenstown、S=Southampton)
class |  カテゴリ |乗船クラス
who | カテゴリ |男性 or 女性
adult_male |  カテゴリ |成人男性であるかどうか
deck |  カテゴリ |乗船していたデッキ
embark_town |  カテゴリ |出港地
alive |  カテゴリ |生存したかどうか
alone |  カテゴリ |一人であったかどうか

## 前処理

このデータには欠損値が含まれているため、前処理を行う。

In [ ]:
# 欠損値を確認する
data.isnull().sum()

In [ ]:
# age, embarked deck の欠損値を埋める

# 平均値で欠損値を埋める。
data['age'].fillna(data['age'].mean(), inplace=True)

# 最頻値で欠損値を埋める。
data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)
data['embark_town'].fillna(data['embark_town'].mode()[0], inplace=True)

# 結果を確認
data.isnull().sum()

## 基礎集計

In [ ]:
# 基本的な統計情報を表示
print(data.describe())

# 生存者数と死亡者数を集計
survived_counts = data['survived'].value_counts()
print("\n生存者数と死亡者数の集計:")
print(survived_counts)

# 乗客のクラスごとの生存率を計算
survival_rate_by_class = data.groupby('pclass')['survived'].mean()
print("\nクラスごとの生存率:")
print(survival_rate_by_class)

# 性別ごとの生存率を計算
survival_rate_by_gender = data.groupby('sex')['survived'].mean()
print("\n性別ごとの生存率:")
print(survival_rate_by_gender)

# 年齢の分布を表示
print("\n年齢の分布:")
print(data['age'].hist())

## ロジスティック回帰モデルの構築

まず、数値型の項目のみでモデルを作成し精度を確認する。

In [ ]:
# 説明変数と目的変数を選択
# 数値データのみで予測
X = data[[ 'age', 'fare', 'sibsp', 'parch']]
y = data[['survived']]

In [ ]:
# データを学習用と検証用に分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 準備のための手順
scaler = StandardScaler()
scaler.fit(X_train)

# 標準化後のものは別の変数にセットすると良い
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)

# ロジスティック回帰モデルを作成
model = LogisticRegression()

# モデルを訓練
model.fit(X_train_scaled, y_train.values.ravel()  )

# テストデータで予測
y_pred = model.predict(X_val_scaled)

# 精度を計算
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy}')

# 回帰係数を可視化
sns.barplot(x=model.coef_[0], y=X.columns)
plt.show()

In [ ]:
# カテゴリ変数をOne Hot Encoding
data_encoded = pd.get_dummies(data[['embarked', 'class', 'who', 'adult_male', 'alone']], drop_first=True)

# 説明変数と目的変数を設定
X = pd.concat([data[['age', 'fare', 'sibsp', 'parch']], data_encoded], axis=1)
y = data['survived']

In [ ]:
# データを訓練セットとテストセットに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 準備のための手順
scaler = StandardScaler()
scaler.fit(X_train)

# 標準化後のものは別の変数にセットすると良い
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)

# ロジスティック回帰モデルを作成
model = LogisticRegression()

# モデルを訓練
model.fit(X_train_scaled, y_train)

# テストデータで予測
y_pred = model.predict(X_val_scaled)

# 精度を計算
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy}')

# 特徴量の重要度を可視化
sns.barplot(x=model.coef_[0], y=X.columns)
plt.show()

## ハイパーパラメータチューニング

## 正則化パラメータ

ロジスティック回帰には、「正則化パラメータ」があり、
例えば、以下のように指定する。

```
model = LogisticRegression(C=1.0)
```

1.0の部分を変更することで精度をより向上させることが可能である。
なお、今までは指定を省略していたが、デフォルトとして C=1.0 が指定されていた。


正則化パラメータとして、どの値が適切かを調べるためには、
**グリッドサーチ**が用いられる。

そのためには、正則化パラメータの候補を以下のように設定する。

```
param_grid = {'C': [ 0.1, 0.15, 0.2, 0.3, 1, 2]}
```

正則化パラメータは正の値で、多くの場合は 0.1 ～ 100の範囲を候補とする。


候補を設定したら、GridSearchCV を設定し、fitで実行する。

```
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train.values.ravel() )
```

実行した結果、最適なモデルを以下のように得ることができる。
```
best_model = grid_search.best_estimator_
```

In [ ]:
# 必要なライブラリをインポート
from sklearn.model_selection import GridSearchCV

# パラメータグリッドを設定
param_grid = {'C': [ 0.1, 0.15, 0.2, 0.3, 1, 2]}

# GridSearchCVを設定
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)

# グリッドサーチを実行
grid_search.fit(X_train_scaled, y_train.values.ravel() )

# 最適なパラメータとスコアを出力
print(f"Best parameters: {grid_search.best_params_}")
# print(f"Best cross-validation score: {grid_search.best_score_}")

# 最適なパラメータでテストデータを評価
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_train_scaled)
accuracy = accuracy_score(y_train, y_pred)
print(f"Validation set accuracy: {accuracy}")


In [ ]:

# 精度が向上する様子をグラフで示す
scores = grid_search.cv_results_['mean_test_score']
plt.figure(figsize=(10, 6))
sns.lineplot(x=param_grid['C'], y=scores)
plt.xlabel('C (Inverse of regularization strength)')
plt.ylabel('Mean Test Score')
plt.title('Optimizing l2 Penalty with Grid Search')
plt.show()

---
---
---


# その他の参考となるサイト
https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package



https://www.kaggle.com/datasets/yeanzc/telco-customer-churn-ibm-dataset
